In [29]:
import os
from dask.distributed import Client
import dask.array as da
import dask
from azureml.core import Workspace, Experiment, Datastore, Dataset, Environment

C:\Users\feli1\AppData\Local\Continuum\anaconda3\envs\daskaks\lib\site-packages\distributed\client.py:1079: VersionMismatchWarning: Mismatched versions found

blosc
+--------------------------+---------+
|                          | version |
+--------------------------+---------+
| client                   | None    |
| scheduler                | 1.8.3   |
| tcp://10.244.0.21:45885  | 1.8.3   |
| tcp://10.244.1.26:36269  | 1.8.3   |
| tcp://10.244.10.4:46161  | 1.8.3   |
| tcp://10.244.11.4:34155  | 1.8.3   |
| tcp://10.244.12.3:37611  | 1.8.3   |
| tcp://10.244.13.4:39753  | 1.8.3   |
| tcp://10.244.14.2:34961  | 1.8.3   |
| tcp://10.244.15.2:40487  | 1.8.3   |
| tcp://10.244.16.3:36037  | 1.8.3   |
| tcp://10.244.17.3:36509  | 1.8.3   |
| tcp://10.244.18.4:39483  | 1.8.3   |
| tcp://10.244.19.4:34431  | 1.8.3   |
| tcp://10.244.2.26:32879  | 1.8.3   |
| tcp://10.244.23.15:45459 | 1.8.3   |
| tcp://10.244.3.3:42167   | 1.8.3   |
| tcp://10.244.4.4:35155   | 1.8.3   |
| tcp://10.244.4

In [2]:
aks_name=<aks_name>
aks_rg=<aks_resource_group>
aks_subscription_id=<aks_subscription_id>
## kubernets_version="1.15.10"

In [3]:
dir_path = os.getcwd()
kube_config_path = dir_path + '\\config'

## check aks status

In [4]:
!kubectl get services --kubeconfig $kube_config_path

NAME                  TYPE           CLUSTER-IP    EXTERNAL-IP      PORT(S)                      AGE
azureml-fe            LoadBalancer   10.0.37.152   104.45.176.211   80:31391/TCP,443:30295/TCP   23d
azureml-fe-int-http   ClusterIP      10.0.32.90    <none>           9001/TCP                     23d
kubernetes            ClusterIP      10.0.0.1      <none>           443/TCP                      23d


In [5]:
!kubectl get pods --kubeconfig $kube_config_path

NAME                               READY   STATUS    RESTARTS   AGE
azureml-ba-5d44c5d5f4-9qbcj        1/1     Running   0          23d
azureml-fe-6cdf785d8f-n2pbk        2/2     Running   4          23d
azureml-fe-6cdf785d8f-ntm4x        2/2     Running   6          23d
azureml-fe-6cdf785d8f-zw9dv        2/2     Running   2          23d
blobfuse-flexvol-installer-4xzxq   1/1     Running   0          23d
blobfuse-flexvol-installer-5vb67   1/1     Running   0          23d
blobfuse-flexvol-installer-6258j   1/1     Running   0          20d
blobfuse-flexvol-installer-78lhq   1/1     Running   0          20d
blobfuse-flexvol-installer-k8dbx   1/1     Running   0          23d
blobfuse-flexvol-installer-kbmkd   1/1     Running   0          23d
blobfuse-flexvol-installer-mqw4f   1/1     Running   0          21d
blobfuse-flexvol-installer-qgcbb   1/1     Running   0          23d
blobfuse-flexvol-installer-qhjbp   1/1     Running   0          23d
blobfuse-flexvol-installer-wzc9n   1/1     Runni

In [15]:
!helm list --kubeconfig $kube_config_path

NAME     	NAMESPACE	REVISION	UPDATED                              	STATUS  	CHART     	APP VERSION
dask-kube	default  	2       	2020-05-06 22:19:36.7818764 -0700 PDT	deployed	dask-4.1.6	2.11.0     


## start dask kubernet cluster

In [8]:
!helm repo add dask https://helm.dask.org/

"dask" has been added to your repositories


In [9]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "dask" chart repository
...Successfully got an update from the "dask-gateway" chart repository
Update Complete. âŽˆ Happy Helming!âŽˆ 


In [10]:
!helm install dask-kube dask/dask --kubeconfig $kube_config_path

NAME: dask-kube
LAST DEPLOYED: Wed May  6 22:18:05 2020
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for installing DASK, released at name: dask-kube.

To learn more about the release, try:

  $ helm status dask-kube  # information about running pods and this message
  $ helm get dask-kube     # get full Kubernetes specification

This release includes a Dask scheduler, 3 Dask workers, and 1 Jupyter servers.

The Jupyter notebook server and Dask scheduler expose external services to
which you can connect to manage notebooks, or connect directly to the Dask
cluster. You can get these addresses by running the following:

  export DASK_SCHEDULER=$(kubectl get svc --namespace default dask-kube-scheduler -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
  export DASK_SCHEDULER_UI_IP=$(kubectl get svc --namespace default dask-kube-scheduler -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
  export DASK_SCHEDULER_PORT=8786
  export DASK_SCHEDULER_UI_PO

## Start/Update wokers

In [11]:
to_write = f"""
image:
  name: mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04
  pullPolicy: IfNotPresent
worker:
  replicas: 20
  resources:
    limits:
      cpu: 1
      memory: 3G
    requests:
      cpu: 1
      memory: 3G
  env:
    - name: EXTRA_APT_PACKAGES
      value: gcc
    - name: EXTRA_PIP_PACKAGES
      value: adlfs azureml-core azureml-dataprep dask[complete] dask-ml[complete] distributed fastparquet fsspec joblib jupyterlab lz4 mpi4py notebook pyarrow

# We want to keep the same packages on the worker and jupyter environments
jupyter:
  enabled: true
  env:
    - name: EXTRA_APT_PACKAGES
      value: gcc
    - name: EXTRA_PIP_PACKAGES
      value: adlfs azureml-core azureml-dataprep dask[complete] dask-ml[complete] distributed fastparquet fsspec joblib jupyterlab lz4 mpi4py notebook pyarrow
""".lstrip()

with open('update_worker.yaml', 'w') as f:
    f.write(to_write)

In [13]:
!helm upgrade dask-kube dask/dask -f update_worker.yaml --kubeconfig $kube_config_path

Release "dask-kube" has been upgraded. Happy Helming!
NAME: dask-kube
LAST DEPLOYED: Wed May  6 22:19:36 2020
NAMESPACE: default
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
Thank you for installing DASK, released at name: dask-kube.

To learn more about the release, try:

  $ helm status dask-kube  # information about running pods and this message
  $ helm get dask-kube     # get full Kubernetes specification

This release includes a Dask scheduler, 20 Dask workers, and 1 Jupyter servers.

The Jupyter notebook server and Dask scheduler expose external services to
which you can connect to manage notebooks, or connect directly to the Dask
cluster. You can get these addresses by running the following:

  export DASK_SCHEDULER=$(kubectl get svc --namespace default dask-kube-scheduler -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
  export DASK_SCHEDULER_UI_IP=$(kubectl get svc --namespace default dask-kube-scheduler -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
  export

In [22]:
# check pod status
!kubectl get pods --kubeconfig $kube_config_path

NAME                                   READY   STATUS    RESTARTS   AGE
azureml-ba-5d44c5d5f4-9qbcj            1/1     Running   0          23d
azureml-fe-6cdf785d8f-n2pbk            2/2     Running   4          23d
azureml-fe-6cdf785d8f-ntm4x            2/2     Running   6          23d
azureml-fe-6cdf785d8f-zw9dv            2/2     Running   2          23d
blobfuse-flexvol-installer-48vwr       1/1     Running   0          119s
blobfuse-flexvol-installer-4xzxq       1/1     Running   0          23d
blobfuse-flexvol-installer-5vb67       1/1     Running   0          23d
blobfuse-flexvol-installer-6258j       1/1     Running   0          20d
blobfuse-flexvol-installer-78lhq       1/1     Running   0          20d
blobfuse-flexvol-installer-bq6sz       1/1     Running   0          118s
blobfuse-flexvol-installer-gtkml       1/1     Running   0          114s
blobfuse-flexvol-installer-k8dbx       1/1     Running   0          23d
blobfuse-flexvol-installer-kbmkd       1/1     Running   0   

In [ ]:
# check worker logs
!kubectl logs dask-kubenetes-jupyter-6df9fc9544-stqcj --kubeconfig $kube_config_path

In [16]:
!kubectl get services --kubeconfig $kube_config_path

NAME                  TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)                       AGE
azureml-fe            LoadBalancer   10.0.37.152    104.45.176.211   80:31391/TCP,443:30295/TCP    23d
azureml-fe-int-http   ClusterIP      10.0.32.90     <none>           9001/TCP                      23d
dask-kube-jupyter     LoadBalancer   10.0.146.160   52.188.180.186   80:30855/TCP                  3m11s
dask-kube-scheduler   LoadBalancer   10.0.243.121   52.188.181.173   8786:31289/TCP,80:30726/TCP   3m11s
kubernetes            ClusterIP      10.0.0.1       <none>           443/TCP                       23d


## Navigate to Jupyter notebook via link provided

==================================================

## Connect dask client

In [19]:
client = Client('52.188.181.173:8786')

C:\Users\feli1\AppData\Local\Continuum\anaconda3\envs\daskaks\lib\site-packages\distributed\client.py:1079: VersionMismatchWarning: Mismatched versions found

blosc
+--------------------------+---------+
|                          | version |
+--------------------------+---------+
| client                   | None    |
| scheduler                | 1.8.3   |
| tcp://10.244.0.21:45885  | 1.8.3   |
| tcp://10.244.1.24:39525  | 1.8.3   |
| tcp://10.244.1.25:34583  | 1.8.3   |
| tcp://10.244.1.26:36269  | 1.8.3   |
| tcp://10.244.2.24:45367  | 1.8.3   |
| tcp://10.244.2.25:38293  | 1.8.3   |
| tcp://10.244.2.26:32879  | 1.8.3   |
| tcp://10.244.23.15:45459 | 1.8.3   |
| tcp://10.244.46.13:43143 | 1.8.3   |
| tcp://10.244.46.15:35799 | 1.8.3   |
| tcp://10.244.51.12:36077 | 1.8.3   |
| tcp://10.244.51.13:46217 | 1.8.3   |
| tcp://10.244.7.12:44445  | 1.8.3   |
| tcp://10.244.8.13:36225  | 1.8.3   |
+--------------------------+---------+

dask
+--------------------------+---------+
|         

In [26]:
client

Client Scheduler: tcp://52.188.181.173:8786 Dashboard: http://52.188.181.173:8787/status,Cluster Workers: 17 Cores: 17 Memory: 51.00 GB


In [30]:
## Delete a helm deployment
!helm delete dask-kube --kubeconfig $kube_config_path

release "dask-kube" uninstalled


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
